# Data curation

Goal: Process the data from the raw database to make it ready for data analysis. This new database will be called prod

In [1]:
# Data curation
import sqlite3

# Used for pretty printing
import pandas as pd

# Creating empty data base
con = sqlite3.connect("unified.db")
cur = con.cursor()

# Enable REGEX for sqlite
import re


def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

con.create_function("REGEXP", 2, regexp)

# Create new table and check for multiple executions
cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='prod'")
if not cur.fetchone():  # If the fetch returns None, table does not exist
    cur.execute("CREATE TABLE prod AS SELECT * FROM initial")
    print("Table 'prod' created successfully.")
else:
    print("Table 'prod' already exists.")



Table 'prod' already exists.


## Valid sequences

Check wether sequence only contains valid amino acids and no whitespaces or any other letters not being valid AA's.

In [2]:
# This query checks if there are any symbols other the the valid aa code
pd.read_sql_query("""SELECT *
                FROM initial
                WHERE seq REGEXP '[^ARNDCEQGHILKMFPSTWYVX]'; 
""", con)
 # WHERE seq REGEXP '[^ARNDCEQGHILKMFPSTWYV]'; to not include X



,id,name,AB,description,OX,dataset,seq,seq_len
0,L11A004522,LAMP2_L11A004522,1,None,alien,LAMP2.fasta,kklaklallkwllalkklallalkk,25
1,L13A15655,LAMP2_L13A15655,1,None,alien,LAMP2.fasta,kklfkkilkyL,11
2,DRAMP20856,dramp_DRAMP20856,1,None,alien,dramp_antimicrobial.fasta,OWOWOWORPVYOPRPRPPHPRL,22
3,DRAMP20857,dramp_DRAMP20857,1,None,alien,dramp_antimicrobial.fasta,OIOIORPVYOPRPRPPHPRL,20
4,DRAMP21410,dramp_DRAMP21410,1,None,alien,dramp_antimicrobial.fasta,klckivvikvck,12
...,...,...,...,...,...,...,...,...
88,P21986,FLA3_SPIAU,0,Flagellar filament 32 kDa core protein (Fragme...,Spirochaeta aurantia OX=147,uniprot_swissprot.fasta,MIINHNMSAINANRVLGBT,19
89,P21987,FLA4_SPIAU,0,Flagellar filament 31.5 kDa core protein (Frag...,Spirochaeta aurantia OX=147,uniprot_swissprot.fasta,MIINHNMSAINANRVLGBTNADITKDL,27
90,P25072,PA21_MICTM,0,Phospholipase A2 1 (Fragment),Micrurus tener microgalbineus OX=8636,uniprot_swissprot.fasta,SLLBFKBMIEST,12
91,P35707,FLAV_NOSSM,0,Flavodoxin (Fragment),Nostoc sp. (strain MAC) OX=35822,uniprot_swissprot.fasta,SKKIGLFYGTZTGKTESVAEIIDEFGDEVVTLDID,35


Problems found in seq:
* non capitalized letters
* the letter X and B in seq



# The lower case problem
This can be solved pretty easy by just replacing all the lowercase seqences with upper case ones.


In [3]:
cur.execute("""
UPDATE initial
SET seq = UPPER(seq)
WHERE seq != UPPER(seq);
""")

print(cur.fetchall())

[]


# The other constraints
Idea is to add a new col called valid. This either says yes or no including the reason why not.

In [4]:
# create col
cur.execute("""
ALTER TABLE initial
ADD COLUMN valid TEXT DEFAULT 'yes';
""")

# update col
cur.execute("""
UPDATE initial
SET valid = CASE 
    WHEN AB NOT IN (0, 1) THEN 'Invalid AB value'
    WHEN dataset IS NULL THEN 'Dataset is null'
    WHEN LENGTH(seq) < 1 OR LENGTH(seq) > 200 THEN 'Invalid seq length'
    WHEN seq REGEXP '^[ARNDCEQGHILKMFPSTWYVX]+$' THEN valid
    ELSE 'Invalid sequence characters'
END;
""")

# uniqueness check
cur.execute("""
UPDATE initial
SET valid = 'Seq must be unique'
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM initial
    GROUP BY seq
);
""")


# Reasons why data is rejected

In [5]:
pd.read_sql_query("SELECT COUNT(*),valid FROM initial GROUP BY valid", con)

,COUNT(*),valid
0,34,Invalid sequence characters
1,420,Seq must be unique
2,12410,yes


I think we have engough data to just reject the 34 invalid sequences.

# Duplicates
Before we just reject all duplicates we need to check wether they are true duplicates meaning if they are stemming from multiple datasets or just technical replicates.

Definition used here:

**technical replicate:** if dataset and seq is a duplicate -

**duplicate:** if only seq is a duplicate but sources state the same AB

**contradictory duplicate** matching seq different source and different AB 

In [6]:
 def identify_duplicates():   
    # Step 1: Identify Technical Replicates
    cur.execute("""
    UPDATE initial
    SET valid = 'technical replicate'
    WHERE seq IN (
        SELECT seq
        FROM initial
        GROUP BY seq, dataset
        HAVING COUNT(*) > 1
    );
    """)
    
    # Step 2: Identify Duplicates (same seq, multiple datasets, same AB)
    cur.execute("""
    UPDATE initial
    SET valid = 'duplicate'
    WHERE seq IN (
        SELECT seq
        FROM initial
        GROUP BY seq, AB
        HAVING COUNT(DISTINCT dataset) > 1
    ) AND valid != 'technical replicate';
    """)
    
    # Step 3: Identify Contradictory Duplicates (same seq, different AB in different datasets)
    cur.execute("""
    UPDATE initial
    SET valid = 'contradictory duplicate'
    WHERE seq IN (
        SELECT seq
        FROM initial
        GROUP BY seq
        HAVING COUNT(DISTINCT AB) > 1 AND COUNT(DISTINCT dataset) > 1
    ) AND valid NOT IN ('technical replicate', 'duplicate');
    """)


identify_duplicates()
#pd.read_sql_query("SELECT * FROM initial WHERE valid = 'contradictory duplicate' ORDER BY seq;", con).to_excel("contradictions.xlsx")
pd.read_sql_query("SELECT COUNT(*), valid FROM initial GROUP BY valid", con)

,COUNT(*),valid
0,33,Invalid sequence characters
1,812,contradictory duplicate
2,20,duplicate
3,6,technical replicate
4,11993,yes


# Adressing contradictory duplicates
Theese stem from negative data sources (i.e uniprot). We can reject the ones from uniprot.

In [7]:
cur.execute("""
UPDATE initial
SET AB = 1
WHERE dataset = 'uniprot_swissprot.fasta' AND valid = 'contradictory duplicate';
""")

identify_duplicates()
pd.read_sql_query("SELECT COUNT(*), valid FROM initial GROUP BY valid", con)

,COUNT(*),valid
0,33,Invalid sequence characters
1,832,duplicate
2,6,technical replicate
3,11993,yes


This means all duplicates stem from the uniprot data base. We have no other duplicates.

# Adressing Invalid sequences and technical duplicates
we just remove them.


In [8]:
cur.execute("""
DELETE FROM initial
WHERE NOT seq REGEXP '^[ARNDCEQGHILKMFPSTWYVX]+$' OR valid = 'technical replicate';
""")

identify_duplicates()
pd.read_sql_query("SELECT COUNT(*), valid FROM initial GROUP BY valid", con)

,COUNT(*),valid
0,830,duplicate
1,11993,yes


# Merge duplicates
To remove the duplicates but keep the information we will merge theese rows containing the duplicates.

From here on we will work with a new table which enforces data integrity. The table enforces the folloing conditions:


 * only valid amino acid seq
 * AB only being 0 or 1
 * AB cant bo 0
 * dataset must not be null
 * seq length between 1 and 200
 * seq must be unique and not null

I have decided to designate seq as database keys. This enforces uniqueness, non nullabilty and improves lookup performance.

In [9]:
pd.read_sql_query("SELECT * FROM  initial WHERE valid = 'duplicate' ORDER BY seq LIMIT 4;", con)

,id,name,AB,description,OX,dataset,seq,seq_len,valid
0,DRAMP00303,dramp_DRAMP00303,1,None,alien,dramp_antimicrobial.fasta,ACGGGGGDVGSLISASLFDQMLKYRNDPRCCXXGF,35,duplicate
1,P29137,CHI1_CASSA,1,Basic endochitinase CH1 (Fragment),Castanea sativa OX=21020,uniprot_swissprot.fasta,ACGGGGGDVGSLISASLFDQMLKYRNDPRCCXXGF,35,duplicate
2,DRAMP02488,dramp_DRAMP02488,1,None,alien,dramp_antimicrobial.fasta,ADDRNPLEEFRENNYEEFL,19,duplicate
3,P0DI85,OXLA8_DEIAC,1,L-amino-acid oxidase ACTX-8 (Fragment),Deinagkistrodon acutus OX=36307,uniprot_swissprot.fasta,ADDRNPLEEFRENNYEEFL,19,duplicate


In [10]:
# Step 1: Create a new table for the merged results
try: 
    cur.execute(""" DROP TABLE prod; """)
except Exception as e:
    print("")

cur.execute("""
CREATE TABLE IF NOT EXISTS prod (
    id TEXT,
    name TEXT,
    AB INTEGER NOT NULL CHECK (AB IN (0, 1)),
    description TEXT,
    OX TEXT,
    source TEXT NOT NULL,
    seq TEXT PRIMARY KEY CHECK (seq = UPPER(seq) AND seq REGEXP '^[ARNDCEQGHILKMFPSTWYVX]+$'),
    valid TEXT
);
""")



# Step 2: Insert aggregated data into the new table INSERT INTO prod(id, name, AB, description, OX, source, seq, valid)
cur.execute("""
INSERT INTO prod(id, name, AB, description, OX, source, seq, valid)
SELECT 
    GROUP_CONCAT(id, '; ') AS id,
    GROUP_CONCAT(name, '; ') AS name,
    AB,
    GROUP_CONCAT(description, '; ') AS description,
    GROUP_CONCAT(OX, '; ') AS OX,
    GROUP_CONCAT(dataset, '; ') AS source,
    UPPER(seq) AS seq,
    'yes - merged duplicate' AS valid
FROM initial
WHERE valid = 'duplicate' AND seq REGEXP '^[ARNDCEQGHILKMFPSTWYVX]+$'
GROUP BY seq;
""")

In [11]:
pd.read_sql_query("SELECT * FROM  prod;", con)

,id,name,AB,description,OX,source,seq,valid
0,DRAMP00303; P29137,dramp_DRAMP00303; CHI1_CASSA,1,Basic endochitinase CH1 (Fragment),alien; Castanea sativa OX=21020,dramp_antimicrobial.fasta; uniprot_swissprot.f...,ACGGGGGDVGSLISASLFDQMLKYRNDPRCCXXGF,yes - merged duplicate
1,DRAMP02488; P0DI85,dramp_DRAMP02488; OXLA8_DEIAC,1,L-amino-acid oxidase ACTX-8 (Fragment),alien; Deinagkistrodon acutus OX=36307,dramp_antimicrobial.fasta; uniprot_swissprot.f...,ADDRNPLEEFRENNYEEFL,yes - merged duplicate
2,ADAM_0078; P86520,InverPep_ADAM_0078; SBTXA_SOYBN,1,Soybean toxin 27 kDa chain (Fragment),alien; Glycine max OX=3847,InverPep.fasta; uniprot_swissprot.fasta,ADPTFGFTPLGLSEKANLQIMKAYD,yes - merged duplicate
3,ADAM_0131; P81493,InverPep_ADAM_0131; DFAX1_BETVU,1,Defensin-like protein AX1,alien; Beta vulgaris OX=161934,InverPep.fasta; uniprot_swissprot.fasta,AICKKPSKFFKGACGRDADCEKACDQENWPGGVCVPFLRCECQRSC,yes - merged duplicate
4,ADAM_0141; P84644,InverPep_ADAM_0141; CIRF_CHAPA,1,Circulin-F,alien; Chassalia parviflora OX=58431,InverPep.fasta; uniprot_swissprot.fasta,AIPCGESCVWIPCISAAIGCSCKNKVCYR,yes - merged duplicate
...,...,...,...,...,...,...,...,...
410,DRAMP04707; P01476,Myotoxin-A; MYX1_CROVV,1,Myotoxin-A,alien; Crotalus viridis viridis OX=8742,InverPep.fasta; uniprot_swissprot.fasta,YKQCHKKGGHCFPKEKICIPPSSDLGKMDCRWKWKCCKKGSG,yes - merged duplicate
411,DRAMP04712; P63176,Myotoxin-3; MYX3_CROVV,1,Myotoxin-3,alien; Crotalus viridis viridis OX=8742,InverPep.fasta; uniprot_swissprot.fasta,YKRCHKKGGHCFPKTVICLPPSSDFGKMDCRWKWKCCKKGSVNNA,yes - merged duplicate
412,ADAM_6429; P68006,InverPep_ADAM_6429; NPY_RABIT,1,Neuropeptide Y,alien; Oryctolagus cuniculus OX=9986 GN=NPY,InverPep.fasta; uniprot_swissprot.fasta,YPSKPDNPGEDAPAEDMARYYSALRHYINLITRQRY,yes - merged duplicate
413,ADAM_6431; P84071,InverPep_ADAM_6431; ASCL_ALLCG,1,Ascalin (Fragment),alien; Allium cepa var. aggregatum OX=28911,InverPep.fasta; uniprot_swissprot.fasta,YQCGQGG,yes - merged duplicate


830 duplicates successfully merged into 415 concatenated rows.

# Merge the rest

Add valid sequences into valid


In [12]:
cur.execute("""
INSERT INTO prod
SELECT id, name, AB, description, OX, dataset, seq, valid FROM initial
WHERE valid = 'yes';
""")

pd.read_sql_query("SELECT * FROM  prod;", con)

,id,name,AB,description,OX,source,seq,valid
0,DRAMP00303; P29137,dramp_DRAMP00303; CHI1_CASSA,1,Basic endochitinase CH1 (Fragment),alien; Castanea sativa OX=21020,dramp_antimicrobial.fasta; uniprot_swissprot.f...,ACGGGGGDVGSLISASLFDQMLKYRNDPRCCXXGF,yes - merged duplicate
1,DRAMP02488; P0DI85,dramp_DRAMP02488; OXLA8_DEIAC,1,L-amino-acid oxidase ACTX-8 (Fragment),alien; Deinagkistrodon acutus OX=36307,dramp_antimicrobial.fasta; uniprot_swissprot.f...,ADDRNPLEEFRENNYEEFL,yes - merged duplicate
2,ADAM_0078; P86520,InverPep_ADAM_0078; SBTXA_SOYBN,1,Soybean toxin 27 kDa chain (Fragment),alien; Glycine max OX=3847,InverPep.fasta; uniprot_swissprot.fasta,ADPTFGFTPLGLSEKANLQIMKAYD,yes - merged duplicate
3,ADAM_0131; P81493,InverPep_ADAM_0131; DFAX1_BETVU,1,Defensin-like protein AX1,alien; Beta vulgaris OX=161934,InverPep.fasta; uniprot_swissprot.fasta,AICKKPSKFFKGACGRDADCEKACDQENWPGGVCVPFLRCECQRSC,yes - merged duplicate
4,ADAM_0141; P84644,InverPep_ADAM_0141; CIRF_CHAPA,1,Circulin-F,alien; Chassalia parviflora OX=58431,InverPep.fasta; uniprot_swissprot.fasta,AIPCGESCVWIPCISAAIGCSCKNKVCYR,yes - merged duplicate
...,...,...,...,...,...,...,...,...
12403,Q9R4P0,RL29_BREVE,0,Large ribosomal subunit protein uL29 (Fragment),Brevundimonas vesicularis OX=41276 GN=rpmC,uniprot_swissprot.fasta,TKIADLRSQTTDQLSDELLKLXKEQ,yes
12404,Q9R4P1,RS21_BREVE,0,Small ribosomal subunit protein bS21 (Fragment),Brevundimonas vesicularis OX=41276 GN=rpsU,uniprot_swissprot.fasta,VQIFVRDNNVDQALKALK,yes
12405,Q9R4P4,RL29_BREDI,0,Large ribosomal subunit protein uL29 (Fragment),Brevundimonas diminuta OX=293 GN=rpmC,uniprot_swissprot.fasta,TKIADLRSQTVDQLSDXLXKL,yes
12406,Q9R4P6,RS21_BREDI,0,Small ribosomal subunit protein bS21 (Fragment),Brevundimonas diminuta OX=293 GN=rpsU,uniprot_swissprot.fasta,VQIFVXDNNVDQALK,yes


# Final health check

In [13]:
pd.read_sql_query("SELECT COUNT(*), valid FROM prod GROUP BY valid", con)

,COUNT(*),valid
0,11993,yes
1,415,yes - merged duplicate


# Save and commit


In [14]:
cur.execute("DROP TABLE initial")
con.commit()
con.close()